In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 필요한 라이브러리 설치
!pip install langchain-community
!pip install -U langchain-community
!pip install --upgrade langchain
!pip install tiktoken
!pip install langchain-google-genai
!pip install chromadb
!pip install langchain_huggingface
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.8 MB/s eta 0:00:00
 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
import os
import pandas as pd
import bs4
import tiktoken
import json
import re
from collections import defaultdict
from tqdm import tqdm
from IPython.display import clear_output
import time

from langchain.docstore.document import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from ragas import evaluate
from ragas.metrics import context_precision, faithfulness

In [4]:
# gemini
YOUR_API_KEY = ''
os.environ['GOOGLE_API_KEY'] = YOUR_API_KEY

In [5]:
# CSV
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/book_data.csv"
df = pd.read_csv(file_path)

In [6]:
df.columns

Index(['TITLE_NM', 'AUTHR_NM', 'PRC_VALUE', 'BOOK_INTRCN_CN'], dtype='object')

In [7]:
df = df.sample(n=10000, random_state=2025)

In [8]:
# 텍스트 청크 분할
def split_text(text, chunk_size=1000, overlap=100):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

In [9]:
# RAG_DB 설정 - ['TITLE_NM', 'AUTHR_NM', 'PRC_VALUE', 'BOOK_INTRCN_CN'], metadata X
RAG_DB = []

# 데이터프레임 행별로(책별로) 청킹하도록
for index, row in df.iterrows():
    title = row['TITLE_NM']
    authr = row['AUTHR_NM']
    price = row['PRC_VALUE']
    text = row['BOOK_INTRCN_CN']

    # 텍스트 split
    chunks = split_text(text)

    # df 행별로 TAG 붙여서 각 청크를 DB에 추가
    for chunk in chunks:
        RAG_DB.append({
            'Title': title,
            'Authr': authr,
            'Price' : price,
            'Text': chunk
        })
# -> 1행과 2행의 서로다른 책은 각 길이에 상관없이 다른 청크로 들어가도록

In [10]:
# RAG_DB - LangChain의 Document
# metadata는 일단 없이 진행
documents = [
    Document(
        page_content=f"Title: {entry['Title']}\nAuthr: {entry['Authr']}\nPrice: {entry['Price']}\nText: {entry['Text']}"
    ) for entry in RAG_DB
]

In [11]:
# HuggingFace 임베딩 모델
model_name = "intfloat/multilingual-e5-large-instruct"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},  # GPU 사용
    encode_kwargs={"normalize_embeddings": True},
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

sentence_xlm-roberta_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [12]:
!pip install faiss-cpu
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [13]:
from tqdm import tqdm
from langchain_community.vectorstores import FAISS

# 벡터스토어 생성을 위해
text_embedding_pairs = []  # (텍스트, 임베딩) 리스트 -> 임베딩된 벡터와 원본 text가 매치되도록
metadata_list = []  # 메타데이터 리스트 - 옵션으로

for doc in tqdm(documents, desc="Processing Documents", unit="document"):
    embedding = hf_embeddings.embed_query(doc.page_content)  # 청크 임베딩
    text_embedding_pairs.append((doc.page_content, embedding))  # (텍스트, 임베딩) 추가
    metadata_list.append(doc.metadata)

# FAISS 벡터 스토어 생성
vectorstore = FAISS.from_embeddings(
    text_embeddings=text_embedding_pairs,  # (텍스트, 벡터) 튜플 리스트 전달
    metadatas=metadata_list,  # 메타데이터 리스트 추가
    embedding=hf_embeddings  # 임베딩
)

Processing Documents: 100%|██████████| 10000/10000 [05:35<00:00, 29.83document/s]


### Retrieval

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate

# Gemini-1.5-flash 모델
llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)

# FAISS 벡터스토어 retriever 생성
dense_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}  # 검색할 문서 개수 설정
)

In [15]:
# Naive RetrievalQA 체인 기존 dpr 기반의  retrieval
dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_gemini,
    retriever=dense_retriever,
    return_source_documents=True
)

In [16]:
# Advanced Pre‑Retrieval Process
# Advanced Pre‑Retrieval은 final_query_chain을 통해 최종 retrival query generation 에 활용
pre_retrieval_prompt = PromptTemplate.from_template("""
다음 대화 내역과 사용자 최신 질문, 그리고 선호도 정보를 참고하여 아래 JSON 형식으로 출력해줘.
출력 형식:
{{
  "search_probability": <0.0~1.0 사이의 숫자>,
  "search_query": "<검색에 사용할 쿼리 (문자열, 필요한 경우)>",
  "follow_up_question": "<추가 정보 요청 질문 (필요하지 않을 경우 빈 문자열)>"
}}

대화 내역:
{history}

사용자 최신 질문:
"{query}"

사용자 선호도 정보:
{information}

검색 전략:
{strategy}

출력 예시:
{{
  "search_probability": 0.8,
  "search_query": "역사책 추천",
  "follow_up_question": ""
}}
""")
# Advanced Pre‑Retrieval 프롬프트는 final_query_chain에 앞서 내부를 참고
# But, 여기서는 multi-turn 방식의 출력을 위해 아래 free-text 기반 multi-turn 프롬프트를 사용함.

In [17]:
# 멀티턴 프롬프트 – 사용자 선호도 및 대화 히스토리를 기반으로 검색 확률, 기본 검색 쿼리(후에 형식을 대화나 키워드 등으로 변경할 수 있록), 추가 질문을 생성
multi_turn_prompt = PromptTemplate.from_template("""
사용자와의 대화 히스토리는 다음과 같아.:

{history}

사용자의 마지막 질문은 다음과 같아.:
"{query}"

## role :

{impormation}
1. 사용자가 책을 찾는 이유를 아는가?
   1) 사용자는 심심해서 그냥 책을 읽고 싶어함.
   2) 사용자는 추천받은 책을 통해서 정보나 기술을 얻고 싶어함.
   3) 사용자는 흥미, 취미 생활 등의 일환으로 서적을 찾고 싶어함.

2. 사용자가 찾고자 하는 책에 대한 정보를 얼만큼 알고 있는가?
   1) 사용자는 찾고자 하는 책에 대한 어떤 사전 지식도 없음.
   2) 사용자는 특정한 책은 없으나, 카테고리 or 작가 or 관련 책 이름을 말하며 비슷한 책을 추천받고 싶어함.
   3) 사용자는 확고하게 찾고 싶은 책이 존재하며, 해당 책이 없다면 해당 책과 최대한 비슷한 책을 찾고 싶어함.

{if}
사용자의 이전 질문들과 답변들을 종합했을 때, 위 {impormation}을 기반하여 사용자의 선호도를 특정할 수 있다면, 벡터 DB 내에서 아래 {strategy}를 활용하여 검색을 진행해.

{strategy}
1. 대화를 종합하면서 검색을 수행할 확률을 0에서 1 사이로 평가해. 0.6 이상이면 검색을 진행해.
2. 검색이 필요하다면, 사용자의 이전 질문들과 답변들을 30~40단어 이내로 요약해서 벡터 DB 검색에 적절한 쿼리를 생성해.
3. 검색에 활용될 쿼리에는 사용자의 이전 질문들과 답변들 중 카테고리 등과 같은 "핵심 키워드 3~5개"를 포함시키도록 해.
4. 검색에는 사용자의 이전 질문들과 답변들을 "책 추천 기준"을 포함하여 요약해라.
   - 예를 들어: "SF 장르 중에서도 AI 관련 테마를 가진 최신 베스트셀러 추천 or 한국의 근현대사를 최대한 사실적으로 기술하고 있는 역사책 추천"
5. 검색에는 사용자의 이전 질문들과 답변들을 요약할 때 "추천 기준"도 함께 포함하고 고려해서 검색해.
   - 예: "최신 AI 관련 베스트셀러 중에서 평점 4.5 이상인 도서"

{else}
- 아직 사용자의 선호도를 파악하지 못하겠다면, 해당 점수를 올릴 수 있는 방향으로 적절한 보충 질문을 1개 생성해.
- 하지만, 사용자가 목적이 뚜렷한 질문에도 불구하고 중복되는 내용을 3번 이상 답변하거나, 최종적으로 6번 이상의 대화를 나눴다면, 적절한 검색이 이뤄질 수 없음을 언급하며 지금까지의 정보를 활용하여 검색해.

출력 형식 예시:
1. 검색 확률: 0.8
2. 검색 쿼리: "AI 철학 관련 최신 도서"
3. 추가 질문: "AI 철학 관련해서 어떤 주제가 궁금하세요?"
""")
# 멀티턴 프롬프트 체인은 그대로 가져가야함 - 후에 에이전트 or 페르소나를 활용한다면 이러한 방식을 고수해야ㅎ함
search_query_chain = LLMChain(llm=llm_gemini, prompt=multi_turn_prompt)

<ipython-input-17-d36d5630e47c>:45: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  search_query_chain = LLMChain(llm=llm_gemini, prompt=multi_turn_prompt)


In [18]:
# Advanced Post‑Retrieval Process
# Gemini를 활용하여 검색된 문서 재정렬 + 요약하고 최종 답변과 출처를 JSON 형식으로 출력
post_retrieval_prompt = PromptTemplate.from_template("""
다음 문서들에서 사용자 질문 "{query}"에 가장 관련 높은 정보를 재정렬하고, 핵심 내용을 요약하여 하나의 통합된 최종 답변을 작성해줘.
문서들:
{documents}
최종 답변은 아래 JSON 형식으로 출력해줘:
{{
  "final_answer": "<최종 답변 내용>",
  "sources": ["문서 제목1", "문서 제목2", "..."]
}}
""")
# post_retrieval_chain는 generate_answer 쪽 내부에서 생성 & 사용

In [19]:
# Final Query Chain
# 여기가 좀 애매함. 고찰이 필요
final_query_prompt = PromptTemplate.from_template("""
지금까지의 대화 내용을 바탕으로, 사용자의 선호도와 요청을 반영하여 검색에 적절한 최종 쿼리를 생성해줘.
아래 전략을 사용해:
- **쿼리 재작성:** 모호하거나 일관성 없는 표현을 명확하고 통일성 있게 재작성
- **쿼리 확장:** 관련 핵심 키워드(예: 시대, 주제, 서술 방식 등)를 추가하여 검색 범위를 보강
- **쿼리 라우팅:** 사용자의 질문 의도를 정확히 파악하여 적절한 검색 쿼리로 구성
대화 내용:
{history}
기본 검색 쿼리: {fallback}
최종 검색 쿼리:
""")
final_query_chain = LLMChain(llm=llm_gemini, prompt=final_query_prompt)

In [20]:
# robust_parse_json_response 함수 (Advanced 응답 파싱용)
def robust_parse_json_response(response):
    """
    Gemini의 응답이 dict인 경우 그대로 사용하고,
    문자열인 경우 코드 블록 제거 후 JSON 파싱함.
    """
    if isinstance(response, dict):
        if "text" in response and isinstance(response["text"], str):
            response_text = response["text"]
        else:
            return response
    else:
        response_text = response
    response_text = response_text.strip()
    if response_text.startswith("```json"):
        response_text = response_text[len("```json"):].strip()
    if response_text.endswith("```"):
        response_text = response_text[:-3].strip()
    try:
        data = json.loads(response_text)
        return data
    except Exception as e:
        print("JSON 파싱 에러:", e)
        return None

In [21]:
# robust_parse_llm_response 함수 (멀티턴 프롬프트 free text 파싱용)
def robust_parse_llm_response(response_text):
    """
    LLM 응답 텍스트에서 검색 확률, 검색 쿼리, 추가 질문을 추출해라.
    예시 형식:
    1. 검색 확률: 0.8
    2. 검색 쿼리: "AI 철학 관련 최신 도서"
    3. 추가 질문: "AI 철학 관련해서 어떤 주제가 궁금하세요?"
    """
    cleaned_text = re.sub(r'\*\*', '', response_text)
    search_score = None
    search_query = None
    follow_up_question = ""

    score_match = re.search(r"검색\s*확률\s*[:：]\s*([\d\.]+)", cleaned_text)
    if score_match:
        try:
            search_score = float(score_match.group(1))
        except Exception as e:
            print("검색 확률 파싱 에러:", e)

    query_match = re.search(r"검색\s*쿼리\s*[:：]\s*\"(.*?)\"", cleaned_text)
    if query_match:
        search_query = query_match.group(1).strip()

    follow_match = re.search(r"추가\s*질문\s*[:：]\s*\"(.*?)\"", cleaned_text)
    if follow_match:
        follow_up_question = follow_match.group(1).strip()

    return search_score, search_query, follow_up_question

### Generation

In [22]:
# 유저 선호도 수집을 위한 대화 히스토리 저장
user_preferences = defaultdict(list)

# 옵션 : 전체 대화 로그 저장 (누적)
log_history = []  # interactive_multi_turn_qa() 는 실행마다 저장 후, 초기화

In [23]:
def categorize_preference(question, response):
    if "장르" in question or "어떤 책" in question:
        user_preferences["genre"].append(response)
    elif "작가" in question or "좋아하는 작가" in question:
        user_preferences["author"].append(response)
    elif "목적" in question or "이유" in question:
        user_preferences["purpose"].append(response)
    else:
        user_preferences["misc"].append(response)

In [24]:
# 답변 생성 함수 (Post-Retrieval Process 강화: 문서 재정렬, 요약 및 통합)
# 전용 재정렬 모델(pretrained 재정렬 모델이나 BM25 등과 결합한 재정렬 알고리즘)을 사용해 검색 결과의 순위를 다시 매김
# 요약 역시 각 문서별 요약을 수행한 뒤, 이 요약본들을 결합하는 별도의 요약/통합 모델을 활용



# generate_answer 함수: 검색된 문서를 Gemini 기반 Post‑Retrieval으로 처리
def generate_answer(final_search_query, original_query):
    # 1. 벡터 DB에서 최종 쿼리로 문서 검색
    result = RetrievalQA.from_chain_type(
        llm=llm_gemini,
        retriever=dense_retriever,
        return_source_documents=True
    ).invoke(final_search_query)

    initial_answer = result['result']
    retrieved_docs = result['source_documents']

    # 2. 검색된 문서들을 하나의 텍스트로 결합 (상위 문서 내용 사용)
    docs_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # 3. Post‑Retrieval: Gemini를 활용하여 문서 재정렬 및 요약
    post_ret_response = post_retrieval_chain.invoke({
        "query": original_query,
        "documents": docs_text
    })
    print("\n[🔄 Gemini Post‑Retrieval 응답]\n", post_ret_response)
    try:
        post_data = json.loads(post_ret_response.strip())
        final_answer = post_data.get("final_answer", "").strip()
        sources = post_data.get("sources", [])
    except Exception as e:
        print("Post‑Retrieval JSON 파싱 에러:", e)
        final_answer = initial_answer.strip()
        # 출처는 metadata에서 Title을 추출
        sources = [doc.metadata.get('Title', '출처 없음') for doc in retrieved_docs]

    return final_answer, sources

In [25]:
# [변경된 search_and_generate_answer 함수]
# → 멀티턴 프롬프트(search_query_chain)를 사용해 사용자 선호도 및 검색 확률을 점진적으로 파악하고,
#    threshold(검색 확률 ≥ 0.6)가 넘으면 advanced pre‑retrieval(최종 쿼리 생성 → retrieval → post‑retrieval)를 실행.
def search_and_generate_answer(query, query_history):
    while True:
        query_summary = "\n".join(query_history[-5:])  # 최근 5개 대화 요약
        # 멀티턴 프롬프트를 통한 사용자 선호도 조사 및 검색 결정
        search_decision_dict = search_query_chain.invoke({
            "history": query_summary,
            "query": query,
            "if": "✅ 검색이 가능한 경우:",
            "else": "❌ 아직 검색이 불가능한 경우:",
            "impormation": "📌 사용자 선호도 분석:",
            "strategy": "🔍 검색 전략:",
        })
        response_text = search_decision_dict["text"].strip()
        print("\n[🔍 멀티턴 프롬프트 응답]\n", response_text)
        # free-text 파서를 통해 검색 확률, 기본 검색 쿼리, 추가 질문 추출
        search_score, base_search_query, follow_up_question = robust_parse_llm_response(response_text)
        print(f"\n[디버그] 파싱 결과: 검색 확률={search_score}, 기본 검색 쿼리='{base_search_query}', 추가 질문='{follow_up_question}'")

        if search_score is None:
            print("\n[❌ 응답 파싱 실패: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

        # threshold 조건: 검색 확률이 0.6 이상이고, 기본 검색 쿼리가 존재하면 Advanced RAG 수행
        if search_score >= 0.6 and base_search_query:
            final_query_result = final_query_chain.invoke({
                "history": "\n".join(query_history),
                "fallback": base_search_query
            })
            # final_query_chain의 반환값이 dict일 경우 "text" 키 추출
            if isinstance(final_query_result, dict):
                final_search_query = final_query_result.get("text", "").strip()
            else:
                final_search_query = final_query_result.strip()
            print(f"\n[🔎 최종 검색 쿼리 생성]: {final_search_query}")

            # 벡터 DB 검색 및 초기 답변 생성 (기존 RetrievalQA 체인 사용)
            result = dpr_qa_chain.invoke(final_search_query)
            initial_answer = result['result']
            retrieved_docs = result['source_documents']
            docs_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

            # Advanced Post‑Retrieval: Gemini를 활용해 문서 재정렬 및 요약 진행
            post_retrieval_chain = LLMChain(llm=llm_gemini, prompt=post_retrieval_prompt)
            post_ret_response = post_retrieval_chain.invoke({
                "query": query,
                "documents": docs_text
            })
            print("\n[🔄 Gemini Post‑Retrieval 응답]\n", post_ret_response)
            post_data = robust_parse_json_response(post_ret_response)
            if post_data is not None:
                final_answer = post_data.get("final_answer", "").strip()
                sources = post_data.get("sources", [])
            else:
                final_answer = initial_answer.strip()
                sources = [doc.metadata.get('Title', '출처 없음') for doc in retrieved_docs]
            if sources:
                book_info = "\n".join([f"- {title}" for title in sources])
                answer_with_info = f"{final_answer}\n\n[📚 책 정보]\n{book_info}"
                print("\n[📚 책 정보]\n", book_info)
            else:
                answer_with_info = final_answer
            return answer_with_info

        # 만약 follow_up_question이 있으면 보충 질문 진행
        if follow_up_question:
            print(f"\n[🤖 보충 질문: {follow_up_question}]")
            query_history.append(f"AI: {follow_up_question}")
            user_response = input("\n사용자 응답: ")
            query_history.append(f"사용자: {user_response}")
            categorize_preference(follow_up_question, user_response)
            print("\n[📚 사용자 선호도 업데이트 완료!]")
            query = f"{query} {follow_up_question} {user_response}"
            continue

        # 검색 확률 낮거나 기본 검색 쿼리가 없으면 추가 정보 요청
        if search_score < 0.6 or not base_search_query:
            print("\n[❌ 검색 확률 낮거나 검색 쿼리 없음: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

In [26]:
# 멀티턴 대화 실행 함수
def interactive_multi_turn_qa():
    query_history = []  # 매 실행마다 초기화
    while True:
        clear_output(wait=True)
        print("📚 멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)")
        print("-" * 50)
        query = input("질문을 입력하세요: ")
        if query.lower() == 'quit':
            print("\n[📝 대화 저장 중...]")
            log_history.append(query_history)
            print("대화가 저장되었습니다. 프로그램을 종료합니다.")
            break
        query_history.append(f"사용자: {query}")
        answer = search_and_generate_answer(query, query_history)
        print("\n[💬 AI의 답변]")
        print(answer)
        query_history.append(f"AI: {answer}")
        input("\n-> 계속하려면 Enter를 누르세요...")

In [28]:
# 실행
interactive_multi_turn_qa()

📚 멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)
--------------------------------------------------
질문을 입력하세요: 역사책을 찾고 있어

[🔍 멀티턴 프롬프트 응답]
 1. **검색 확률:** 0.4

2. **검색 쿼리:** (아직 검색 쿼리를 생성할 수 없습니다.)

3. **추가 질문:** 어떤 종류의 역사책을 찾으시나요? (예: 시대, 지역, 주제, 특정 사건, 작가 등)


**설명:**

사용자는 "역사책을 찾고 있다"는 정보만 제공했습니다.  📌 사용자 선호도 분석의 1번과 2번 질문에 대한 답을 얻을 수 없기 때문에 검색 확률이 낮습니다.  추가 질문을 통해 사용자가 원하는 역사책의 종류(시대, 지역, 주제 등)에 대한 정보를 얻어야 검색 쿼리를 생성하고 검색을 진행할 수 있습니다.  현재 정보만으로는 사용자의 선호도를 특정할 수 없기 때문에  검색 쿼리를 생성하지 않았습니다.

[디버그] 파싱 결과: 검색 확률=0.4, 기본 검색 쿼리='None', 추가 질문=''

[❌ 검색 확률 낮거나 검색 쿼리 없음: 추가 정보 필요]
추가 정보를 입력해주세요: 한국사. 특히 조선시대 관련된 책이면 좋겠어

[🔍 멀티턴 프롬프트 응답]
 1. **검색 확률:** 0.9

2. **검색 쿼리:** "조선시대 한국사, 역사책 추천, 사실적 서술"

3. **추가 질문:**  (필요 없음)


**설명:**

📌 사용자 선호도 분석:

1. 사용자가 책을 찾는 이유: 3) 사용자는 흥미, 취미 생활 등의 일환으로 서적을 찾고 싶어함. (역사 공부를 하고 싶어하는 것으로 추정됨)

2. 사용자가 찾고자 하는 책에 대한 정보: 2) 사용자는 특정한 책은 없으나, 카테고리(한국사, 조선시대)를 말하며 비슷한 책을 추천받고 싶어함.


🔍 검색 전략 적용:

사용자는 한국사, 특히 조선시대 관련 역사책을 찾고 있다는 것을 명확히 밝혔습니다.  추가적인 정보는 없지만,  목적이 명확하고 질문이 간결하여 검

KeyboardInterrupt: Interrupted by user